## Import libraries and data

In [18]:
%run load_data.py

/anaconda/lib/python3.6/site-packages/IPython/core/magics/execution.py:728: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  exit_ignore=exit_ignore)
/anaconda/lib/python3.6/site-packages/IPython/core/magics/execution.py:728: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  exit_ignore=exit_ignore)
/anaconda/lib/python3.6/site-packages/IPython/core/magics/execution.py:728: DtypeWarning: Columns (0,112) have mixed types. Specify dtype option on import or set low_memory=False.
  exit_ignore=exit_ignore)
/anaconda/lib/python3.6/site-packages/IPython/core/magics/execution.py:728: DtypeWarning: Columns (0,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  exit_ignore=exit_ignore)
/anaconda/lib/python3.6/site-packages/IPython/core/magics/execution.py:728: DtypeWarning: Columns (0) have mixed types. Specify dtype o

## Separate Regressions for Each Loan Grade

In [10]:
# Regressions of rates on personal characteristics

from sklearn.linear_model import LogisticRegression

train_features_no_int = train_features.drop("int_rate_num", 1)
test_features_no_int = test_features.drop("int_rate_num", 1)

for grade in ["A", "B", "C", "D", "E", "F", "G"]: 
    
    lr_fail_rate = LogisticRegression()
    lr_fail_rate.fit(train_features_no_int[train_labels["grade_" + grade] == 1], 
                     train_labels[train_labels["grade_" + grade] == 1]["bad"])
    test_features["pred_fail_rate_" + grade] = lr_fail_rate.predict_proba(test_features_no_int)[:, 1]
    
    mask = (train_labels["grade_" + grade] == 1) & (train_labels["bad"] == 1)
    lr_rec_rate = sm.OLS(train_labels[mask]["return"], sm.add_constant(train_features_no_int[mask])).fit()
    test_features["pred_rec_rate_" + grade] = lr_rec_rate.predict(test_features_no_int)
    
    lr_rec_rate2 = sm.OLS(train_labels[mask]["return"] ** 2, sm.add_constant(train_features_no_int[mask])).fit()
    test_features["pred_rec_rate2_" + grade] = lr_rec_rate2.predict(test_features_no_int)
    
    r = np.mean(train_features[train_labels["grade_" + grade] == 1]["int_rate_num"])
    
    test_features["exp_R_" + grade] = ((1 + r) * (1 - test_features["pred_fail_rate_" + grade]) +
                         test_features["pred_rec_rate_" + grade] * test_features["pred_fail_rate_" + grade])
    
    test_features["var_R_" + grade] = ((1 + r)**2 * (1 - test_features["pred_fail_rate_" + grade]) +
                         test_features["pred_rec_rate2_" + grade] * test_features["pred_fail_rate_" + grade] 
                         - test_features["exp_R_" + grade]**2)
    
    test_features["var_R_" + grade + "_H"] = test_features["var_R_" + grade] * (test_features["var_R_" + grade] >= 0.02)
    test_features["var_R_" + grade + "_L"] = test_features["var_R_" + grade] * (test_features["var_R_" + grade] < 0.02)

In [11]:
np.median(test_features[["var_R_" + g for g in ["A", "B", "C", "D", "E", "F", "G"]]])

0.022811416681872254

## ML Discrete Choice Model

In [12]:
def nllh(beta, features, labels):
    utilities = {}
    for grade in ["A", "B", "C", "D", "E", "F", "G"]:
        utilities[grade] = np.dot(features[["exp_R_" + grade, "var_R_" + grade]].values, beta)
    utilitiesArray = pd.DataFrame(utilities).values
    indicatorArray = labels[["grade_" + x for x in ["A", "B", "C", "D", "E", "F", "G"]]].values
    term1 = np.sum(utilitiesArray * indicatorArray, axis=1)
    term2 = np.log(np.sum(np.exp(utilitiesArray), axis=1))
    return -1 * np.sum(term1 - term2)

def grad(beta, features, labels):
    X1, X2 = {}, {}
    utilities = {}
    for grade in ["A", "B", "C", "D", "E", "F", "G"]:
        X1[grade] = features["exp_R_" + grade].values
        X2[grade] = features["var_R_" + grade].values
        utilities[grade] = np.dot(features[["exp_R_" + grade, "var_R_" + grade]].values, beta)
    X1Array = pd.DataFrame(X1).values
    X2Array = pd.DataFrame(X2).values
    utilitiesArray = pd.DataFrame(utilities).values
        
    indicatorArray = labels[["grade_" + x for x in ["A", "B", "C", "D", "E", "F", "G"]]].values
    
    d1 = (np.sum(X1Array * indicatorArray, axis=1) - 
     np.sum(np.exp(utilitiesArray) * X1Array, axis=1) / np.sum(np.exp(utilitiesArray), axis=1))
    d2 = (np.sum(X2Array * indicatorArray, axis=1) - 
     np.sum(np.exp(utilitiesArray) * X2Array, axis=1) / np.sum(np.exp(utilitiesArray), axis=1))
    
    return -1 * np.array([np.sum(d1), np.sum(d2)])

In [13]:
def nllh2(beta, features, labels):
    utilities = {}
    for grade in ["A", "B", "C", "D", "E", "F", "G"]:
        utilities[grade] = np.dot(features[["exp_R_" + grade, "var_R_" + grade + "_H", "var_R_" + grade + "_L"]].values, beta)
    utilitiesArray = pd.DataFrame(utilities).values
    indicatorArray = labels[["grade_" + x for x in ["A", "B", "C", "D", "E", "F", "G"]]].values
    term1 = np.sum(utilitiesArray * indicatorArray, axis=1)
    term2 = np.log(np.sum(np.exp(utilitiesArray), axis=1))
    return -1 * np.sum(term1 - term2)

def grad2(beta, features, labels):
    X1, X2, X3 = {}, {}, {}
    utilities = {}
    for grade in ["A", "B", "C", "D", "E", "F", "G"]:
        exp_R = features["exp_R_" + grade]
        var_R_H = (features["var_R_" + grade]) * (features["var_R_" + grade] >= 0.02)
        var_R_L = (features["var_R_" + grade]) * (features["var_R_" + grade] < 0.02)
        X = np.array([exp_R, var_R_H, var_R_L])
        X1[grade] = exp_R.values
        X2[grade] = var_R_H.values
        X3[grade] = var_R_L.values
        utilities[grade] = np.dot(X.T, beta)
    X1Array = pd.DataFrame(X1).values
    X2Array = pd.DataFrame(X2).values
    X3Array = pd.DataFrame(X3).values
    utilitiesArray = pd.DataFrame(utilities).values
    indicatorArray = labels[["grade_" + x for x in ["A", "B", "C", "D", "E", "F", "G"]]].values
    
    d1 = (np.sum(X1Array * indicatorArray, axis=1) - 
     np.sum(np.exp(utilitiesArray) * X1Array, axis=1) / np.sum(np.exp(utilitiesArray), axis=1))
    d2 = (np.sum(X2Array * indicatorArray, axis=1) - 
     np.sum(np.exp(utilitiesArray) * X2Array, axis=1) / np.sum(np.exp(utilitiesArray), axis=1))
    d3 = (np.sum(X3Array * indicatorArray, axis=1) - 
     np.sum(np.exp(utilitiesArray) * X3Array, axis=1) / np.sum(np.exp(utilitiesArray), axis=1))
    
    return -1 * np.array([np.sum(d1), np.sum(d2), np.sum(d3)])

In [14]:
import scipy.optimize

scipy.optimize.minimize(nllh2, [0,0,0], args=(test_features, test_labels), method='L-BFGS-B')

      fun: 817935.85271074797
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.       , -0.0349246,  0.0349246])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 60
      nit: 12
   status: 0
  success: True
        x: array([ -2.62191501, -41.14262413,   0.14759032])

## Bootstrapping

In [21]:
train_combined = pd.concat([train_features, train_labels], axis=1)
test_combined = pd.concat([test_features, test_labels], axis=1)

from sklearn.utils import resample
import random
import scipy.optimize

def bootstrap(train_combined, test_combined, iterations, results):
    for i in range(iterations):
        bs_train_combined = resample(train_combined)
        bs_train_features = bs_train_combined.iloc[:, :train_features.shape[1]]
        bs_train_labels = bs_train_combined.iloc[:, train_features.shape[1]:]

        bs_test_combined = resample(test_combined)
        bs_test_features = bs_test_combined.iloc[:, :test_features.shape[1]]
        bs_test_labels = bs_test_combined.iloc[:, test_features.shape[1]:]

        predict(bs_train_features, bs_train_labels, bs_test_features)

        opt = scipy.optimize.minimize(nllh2, [0,0,0], args=(bs_test_features, bs_test_labels), 
                                      method='L-BFGS-B')
        results.append(opt.x)
        
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

def predict(train_features, train_labels, test_features):
    train_features_no_int = train_features.drop("int_rate_num", 1)
    test_features_no_int = test_features.drop("int_rate_num", 1)

    for grade in ["A", "B", "C", "D", "E", "F", "G"]: 

        lr_fail_rate = LogisticRegression()
        lr_fail_rate.fit(train_features_no_int[train_labels["grade_" + grade] == 1], 
                         train_labels[train_labels["grade_" + grade] == 1]["bad"])
        test_features["pred_fail_rate_" + grade] = lr_fail_rate.predict_proba(test_features_no_int)[:, 1]

        mask = (train_labels["grade_" + grade] == 1) & (train_labels["bad"] == 1)
        lr_rec_rate = sm.OLS(train_labels[mask]["return"], sm.add_constant(train_features_no_int[mask])).fit()
        test_features["pred_rec_rate_" + grade] = lr_rec_rate.predict(test_features_no_int)

        lr_rec_rate2 = sm.OLS(train_labels[mask]["return"] ** 2, sm.add_constant(train_features_no_int[mask])).fit()
        test_features["pred_rec_rate2_" + grade] = lr_rec_rate2.predict(test_features_no_int)

        r = np.mean(train_features[train_labels["grade_" + grade] == 1]["int_rate_num"])

        test_features["exp_R_" + grade] = ((1 + r) * (1 - test_features["pred_fail_rate_" + grade]) +
                             test_features["pred_rec_rate_" + grade] * test_features["pred_fail_rate_" + grade])

        test_features["var_R_" + grade] = ((1 + r)**2 * (1 - test_features["pred_fail_rate_" + grade]) +
                             test_features["pred_rec_rate2_" + grade] * test_features["pred_fail_rate_" + grade] 
                             - test_features["exp_R_" + grade]**2)
        
        test_features["var_R_" + grade + "_H"] = test_features["var_R_" + grade] * (test_features["var_R_" + grade] >= 0.02)
        test_features["var_R_" + grade + "_L"] = test_features["var_R_" + grade] * (test_features["var_R_" + grade] < 0.02)
    return

In [24]:
# random.seed(0)
iterations = 6
# results = []

bootstrap(train_combined, test_combined, iterations, results)

In [25]:
results

[array([ -3.82909317, -40.31327598,   2.25964463]),
 array([ -4.84750167, -39.20883761,   3.79547456]),
 array([ -2.88291253, -41.17606688,  -0.71716199]),
 array([ -1.22658292, -42.62568441,  -3.46545261]),
 array([ -4.32221539, -39.82696137,   2.26231886]),
 array([ -7.00150648, -37.45627132,   9.46850808]),
 array([ -3.69381885, -39.97997708,   3.30526353]),
 array([  0.82687811, -42.50608532,  -3.21644409]),
 array([ -4.25800463, -39.86547566,   3.7137405 ]),
 array([ -3.51852037, -40.42214923,   2.50883993])]